In [1]:
import numpy as np
import pandas as pd
import math
import IPython

import pygal
from pygal.style import DarkStyle

import importlib
import svgwrite

def svgreload():
    !osascript -e 'tell application "Google Chrome" to tell the active tab of second window to reload'

In [ ]:
from pygal.style import Style
custom_style = Style(
    background = 'black',
    plot_background = '#111111',
    foreground = '#999999',
    foreground_strong = '#eeeeee',
    foreground_subtle = '#555555',
    opacity = '.8',
    opacity_hover = '.4',
    transition = '10ms',
    colors = ('#B400FF', '#3232FF'),
    label_font_size = 24,
    major_label_font_size = 24,
    title_font_size = 24,
    legend_font_size = 18
    
)

In [36]:
names = !head -1 ~/Dropbox/LiLab/RNAseq/CSV/zfUVvsS_all.csv
gene = !grep opn1sw1 ~/Dropbox/LiLab/RNAseq/CSV/zfUVvsS_all.csv
type(gene)
# pd.DataFrame(data=data[0],columns=names[0])
gdf=pd.DataFrame(data=gene)[0].str.split(',',expand=True)
ndf=pd.DataFrame(data=names)[0].str.split(',',expand=True) 
ndf

IPython.utils.text.SList

In [60]:
!head -1 ~/Dropbox/LiLab/RNAseq/CSV/zfUVvsS_all.csv > ~/Dropbox/LiLab/RNAseq/CSV/temp.txt
!grep -w rho ~/Dropbox/LiLab/RNAseq/CSV/zfUVvsS_all.csv >> ~/Dropbox/LiLab/RNAseq/CSV/temp.txt
gdf = pd.read_csv("~/Dropbox/LiLab/RNAseq/CSV/temp.txt")
gdf
# \

# grep ^.*@.*fish.*$

,symbol,genename,baseMean,log2FoldChange,lfcSE,pvalue,padj,UV01,UV02,UV03,UV04,S01,S02,S03,S04,chr-rank
0,gabrr3a,"gamma-aminobutyric acid (GABA) A receptor, rho 3a",11.81,-0.1694,0.9730,0.614828,0.999194,0,0,27,0,0,29,20,17,5206
1,rho,rhodopsin,207561.67,-0.0154,0.7499,0.976022,0.999194,395257,106284,66731,252358,367166,93345,272102,107250,9938
2,gabrr2a,"gamma-aminobutyric acid (GABA) A receptor, rho 2a",34.91,0.1464,0.9957,0.612222,0.999194,26,186,0,0,0,0,68,0,15603
3,gstr,glutathione S-transferase rho,281.96,0.4081,1.0452,0.303268,0.999194,889,426,440,94,48,357,0,0,26994


### According to Neuhauss recoverin1a in enriched in rods and UV-cones

In [ ]:
hist = pygal.Histogram(
    x_title='',
    y_title='read counts (RNAseq)',
    show_x_labels=False,
    title='UV-opsin (opn1sw1)',
    style=custom_style,
    legend_at_bottom=True,
    legend_at_bottom_columns=4,
    show_legend=False
)

hist.add('UV-Cone',[
    (1831890.775,0.2,0.8),
    (1940418.738,1.2,1.8),
    (1791047.718,2.2,2.8),
    (2139963.138,3.2,3.8),
])
hist.add('S-Cone',[
    (253221.171,4.2,4.8),
    (1152.782,5.2,5.8),
    (39185.840,6.2,6.8),
    (293059.944,7.2,7.8),
])


hist.render_to_file('opn1sw1.svg')

In [ ]:
hist = pygal.Histogram(
    x_title='',
    y_title='read counts (RNAseq)',
    show_x_labels=False,
    title='S-opsin (opn1sw2)',
    style=custom_style,
    legend_at_bottom=True,
    legend_at_bottom_columns=4,
    show_legend=False
)

hist.add('UV-Cone',[
    (6178.852,0.2,0.8),
    (6129,1.2,1.8),
    (109067,2.2,2.8),
    (1407,3.2,3.8),
])
hist.add('S-Cone',[
    (671153.171,4.2,4.8),
    (740768.782,5.2,5.8),
    (885780.840,6.2,6.8),
    (847799.944,7.2,7.8),
])


hist.render_to_file('opn1sw2.svg')

In [ ]:
hist = pygal.Histogram(
    x_title='',
    y_title='read counts (RNAseq)',
    show_x_labels=False,
    title='rhodopsin (rho)',
    style=custom_style,
    legend_at_bottom=True,
    legend_at_bottom_columns=4,
    show_legend=False,
    range=(0,800000)
)

hist.add('UV-Cone',[
    (395257,0.2,0.8),
    (106284,1.2,1.8),
    (66731,2.2,2.8),
    (252358,3.2,3.8),
])
hist.add('S-Cone',[
    (367167,4.2,4.8),
    (93345,5.2,5.8),
    (272102,6.2,6.8),
    (107250,7.2,7.8),
])


hist.render_to_file('test.svg')

In [ ]:
hist = pygal.Histogram(
    x_title='',
    y_title='read counts (RNAseq)',
    show_x_labels=False,
    title='cone transducin (gnat2) vs. rod transducin (gnat1)',
    style=custom_style,
    legend_at_bottom=True,
    legend_at_bottom_columns=4,
    show_legend=False,
#     range=(0,800000)
)

hist.add('UV-Cone',[
    (340844,0.2,0.8),
    (453770,1.2,1.8),
    (434098,2.2,2.8),
    (481402,3.2,3.8),
])
hist.add('S-Cone',[
    (446590,4.2,4.8),
    (362607,5.2,5.8),
    (318034,6.2,6.8),
    (413598,7.2,7.8),
])


hist.add('UV-Cone',[
    (115155,9.2,9.8),
    (39631,10.2,10.8),
    (46918,11.2,11.8),
    (92691,12.2,12.8),
])
hist.add('S-Cone',[
    (167981,13.2,13.8),
    (59011,14.2,14.8),
    (97955,15.2,15.8),
    (33217,16.2,16.8),
])

hist.render_to_file('gnats.svg')




In [ ]:
hist = pygal.Histogram(
    x_title='',
    y_title='million reads',
    show_x_labels=False,
    title='',
    style=custom_style,
    legend_at_bottom=True,
    legend_at_bottom_columns=4,
    show_legend=False
)

hist.add('UV-Cone',[
    (15.710882,0.2,0.8),
    (10.873172,1.2,1.8),
    (17.410638,2.2,2.8),
    (15.763850,3.2,3.8),
])
hist.add('S-Cone',[
    (15.390731,4.2,4.8),
    (15.557118,5.2,5.8),
    (15.617877,6.2,6.8),
    (24.908813,7.2,7.8),
])



hist.render_to_file('test.svg')

In [ ]:
hist = pygal.Histogram(
    x_title='',
    y_title='read counts (RNAseq)',
    show_x_labels=False,
    title='nrxn3',
    style=custom_style,
    legend_at_bottom=True,
    legend_at_bottom_columns=4
)

hist.add('S-Cone',[
    (2823,0.2,0.8),
    (2810,1.2,1.8),
    (2430,2.2,2.8),
    (4880,3.2,3.8),
    (2088,4.2,4.8)
])
hist.add('S-Cone(hib)',[
    (1420,5.2,5.8),
    (951,6.2,6.8),
])
hist.add('M-Cone',[
    (0,7.2,7.8),
    (1,8.2,8.8),
    (2,9.2,9.8)
])
hist.add('M-Cone(hib)',[
    (1,10.2,10.8),
    (1,11.2,11.8),
    (1,12.2,12.8),
])

hist.render_to_file('test.svg')